# Visualization

Work in progress ...

The goal is to visualize g-code path in the jupyter notebook.

## Setup

In [ ]:
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://github.com/dzufferey/my_mvn_repo/raw/master/repository"))

In [ ]:
import $ivy.`io.github.dzufferey::libgcode:0.1-SNAPSHOT`
import $ivy.`com.lihaoyi::scalatags:0.8.2`

## Declarations

The plan is to use [X3DOM](https://www.x3dom.org/) for the vizualization.
For rapid prototyping is quite good.

To make the code readable, first the tags and attributes of the subset of X3Dom which will be used.

In [ ]:
import scalatags.Text.all._
import scalatags.stylesheet._
import almond.display.{Html, Text}

object X3D {
    
    //tags
    lazy val x3d = tag("x3d")
    lazy val scene = tag("scene")
    lazy val viewpoint = tag("viewpoint")
    lazy val orthoviewpoint = tag("orthoviewpoint")
    lazy val navigationInfo = tag("navigationinfo")
    lazy val transform = tag("transform")
    lazy val group = tag("group")
    lazy val billboard = tag("billboard")
    lazy val shape = tag("shape")
    lazy val lineSet = tag("lineset")
    lazy val indexedLineSet = tag("indexedlineset")
    lazy val indexedFaceSet = tag("indexedfaceset")
    lazy val plane = tag("plane")
    lazy val sphere = tag("sphere")
    lazy val cone = tag("cone")
    lazy val text = tag("text")
    lazy val fontStyle = tag("fontstyle")
    lazy val coordinate = tag("coordinate")
    lazy val appearance = tag("appearance")
    lazy val material = tag("material")
    lazy val depthMode = tag("depthMode")
    lazy val lineProperties = tag("lineproperties")
    
    //attributes
    lazy val use = attr("use")
    lazy val defn = attr("def")
    lazy val orientation = attr("orientation")
    lazy val position = attr("position")
    lazy val typeParams = attr("typeparams")
    lazy val rotation = attr("rotation")
    lazy val translation = attr("translation")
    lazy val radius = attr("radius")
    lazy val primType = attr("primtype")
    lazy val subdivision = attr("subdivision")
    lazy val vertexCount = attr("vertexcount")
    lazy val coordIndex = attr("coordindex")
    lazy val index = attr("index")
    lazy val point = attr("point")
    lazy val solid = attr("solid")
    lazy val string = attr("string")
    lazy val colorPerVertex = attr("colorpervertex")
    lazy val lit = attr("lit")
    //lazy val color = attr("color")
    lazy val emissiveColor = attr("emissivecolor")
    lazy val diffuseColor = attr("diffusecolor")
    lazy val specularColor = attr("specularcolor")
    lazy val ambientIntensity = attr("ambientintensity")
    lazy val lineWidthScaleFactor = attr("linewidthscalefactor")
    lazy val readOnly = attr("readonly")

    
    def lineAppearance(color: String, scale: Double = 1.0) = {
        appearance(
            material( diffuseColor := "0 0 0", specularColor := "0 0 0", emissiveColor := color ),
            lineProperties( lineWidthScaleFactor := scale.toString )
        )
    }
    
}

class X3DViewer(uid: Long = scala.util.Random.nextLong(Long.MaxValue)) {
    
    import X3D.{position => xpos, _}
    
    val defaultRotation = "1 0 0 -1.570796326795" // z axis up
    
    val mainViewer = "mainViewer" + uid.toString
    val sidePanelId = "sidePanel" + uid.toString
    val gridAndAxes = "gridAndAxes" + uid.toString
    val gridCheckbox = "gridCheckbox" + uid.toString
    val gridInner = "gridInner" + uid.toString
    val gridBorder = "gridBorder" + uid.toString
  
    def toggleVisibility(id: String) = {
        s"""{var shape = document.getElementById('$id');
            |if (this.checked) shape.setAttribute('render', true);
            |else shape.setAttribute('render', false);}""".stripMargin
    }
    
    def updateGrid(size: String, tick: String) = {
        s"""{var plane = document.getElementById('$gridInner');
            |var s = Math.ceil($size/$tick) * $tick;
            |plane.size=(2*s) + " " + (2*s);
            |plane.subdivision=(2*s/$tick) + " " + (2*s/$tick);
            |var border = document.getElementById('$gridBorder');
            |border.point = (-s) + ' ' + (-s) + ' 0.0 , ' +
            |               (-s) + ' ' + ( s) + ' 0.0 , ' +
            |               ( s) + ' ' + ( s) + ' 0.0 , ' +
            |               ( s) + ' ' + (-s) + ' 0.0';}""".stripMargin
    }
    
    //TODO togglePerspective: center of rotation/position does not work and adapt field of view
    def togglePerspective = {
        s"""{var runtime = document.getElementById('$mainViewer').runtime;
            |var old_vp = runtime.viewpoint();
            |runtime.nextView();
            |var new_vp = runtime.viewpoint();
            |new_vp.centerOfRotation = old_vp.centerOfRotation; 
            |new_vp.orientation = old_vp.orientation;
            |new_vp.position = old_vp.position;}""".stripMargin
    }
    
    def sidePanel = {
        div(id := sidePanelId,
            top := "0",
            right := "-10px",
            zIndex := "2",
            backgroundColor := "Black",
            color := "White",
            transition := "0.5s",
            height := "15%",
            width := "0",
            position := "absolute",
            overflowX := "hidden",
            margin := "10px",
        )(
            div(margin := "5px")(
                button(onclick := s"document.getElementById('$sidePanelId').style.width = '0%';")("close")
            ),
            div(margin := "5px")(
                input(id := gridCheckbox, tpe := "checkbox", checked := "checked", onchange := toggleVisibility(gridAndAxes)),
                label( `for` := gridCheckbox)("Grid")
            ),
            div(margin := "5px")(
                button(onclick := togglePerspective)("Perspective")
            ),
        )
    }
    
    
    def grid(_size: Int, step: Int) = {
        val grey = appearance( material( diffuseColor := "0.0 0.0 0.0", specularColor := "0.0 0.0 0.0", emissiveColor := "0.3 0.3 0.3" ) )
        val sSize = s"${2*_size} ${2*_size}"
        val sStep = s"${2*_size/step} ${2*_size/step}"
        val sBorder = s"-${_size} -${_size} 0.0 , -${_size} ${_size} 0.0 , ${_size} ${_size} 0.0, ${_size} -${_size} 0.0"
        group(
            shape(
                plane(id := gridInner, solid := "false", size := sSize, primType := "LINES", subdivision := sStep ),
                grey
            ),
            shape(
                indexedLineSet(coordIndex := "0 1 2 3 0", colorPerVertex := "false", lit := "false")(
                    coordinate(id := gridBorder, point := sBorder)
                ),
                grey
            )
        )
    }
    
    def axes(length: Int, step: Int) = {
        group(
            shape( // X
                lineSet( vertexCount := "2" ) (
                    coordinate( point := s"0 0 0.001, $length 0 0.001")
                ),
                lineAppearance("1 0 0")
            ),
            shape( // Y
                lineSet( vertexCount := "2" ) (
                    coordinate( point := s"0 0 0.001, 0 $length 0.001")
                ),
                lineAppearance("0 1 0")
            ),
            shape( // Z
                lineSet( vertexCount := "2" ) (
                    coordinate( point := s"0 0 0, 0 0 $length")
                ),
                lineAppearance("0 0 1")
            ),
            for (z <- 0 to length by step) yield {
                shape(
                    lineSet( vertexCount := "2" ) (
                        coordinate( point := s"0 0 $z, $step 0 $z")
                    ),
                    lineAppearance("0 0 1")
                )
            }
        )
    }
    
    //TODO orthographic and normal projection, some script to swtich between the two
    //TODO shortcuts to pick front/back, left/right, top/bottom views
    //TODO pick initial zoom level according to content bounded box
    
    def viewer(content: Modifier) = {
        div(backgroundColor := "rgba(128, 128, 196, 0.4)",
            borderStyle := "solid",
            script( tpe := "text/javascript", src := "https://www.x3dom.org/download/x3dom.js" ),
            link( rel := "stylesheet", tpe := "text/css", href := "https://www.x3dom.org/download/x3dom.css" ),
            x3d(id := mainViewer)(
                scene(
                    viewpoint( id:= "viewPointPersp", xpos := "5.53912 7.69774 6.54642" , orientation := "-0.69862 0.66817 0.25590 1.00294" ),
                    navigationInfo( id := "navi", tpe := "\"TURNTABLE\" \"ANY\"", typeParams := "0, 0, 0.0, 3.14" ),
                    orthoviewpoint( id:= "viewPointOrtho", xpos := "5.53912 7.69774 6.54642" , orientation := "-0.69862 0.66817 0.25590 1.00294" ),
                    transform(rotation := defaultRotation)(
                        group(id := gridAndAxes)(
                            grid(10, 1),
                            axes(100, 1),
                        ),
                        content
                    )
                )
            ),
            button( fontSize := "20px",
                    cursor := "pointer",
                    backgroundColor := "Black",
                    color := "White",
                    padding := "5px 10px",
                    border := "none",
                    position := "absolute",
                    top := "5px",
                    right := "25px",
                    zIndex := "1",
                    onclick := s"document.getElementById('$sidePanelId').style.width = '15%';")("Options"),
            sidePanel
        ).render
    }
    
    //some helpers to create elements
    def display(content: Modifier) =  Html(viewer(content))
    
    def debugDisplay(content: Modifier) =  Text(viewer(content))
    
}

object X3DViewer {
    
    def viewer(content: Modifier) = {
        val v = new X3DViewer()
        v.viewer(content)
    }
    
    def display(content: Modifier) =  Html(viewer(content))
    
    def debugDisplay(content: Modifier) =  Text(viewer(content))
    
}

A simple test

In [ ]:
X3DViewer.display(
    X3D.shape(
        X3D.sphere(X3D.radius := "0.2"),
        X3D.material( X3D.emissiveColor := "0.8 0.8 0.8" )
    )
)

Now that there is a basic viewer, we need to turn "plot" g-code.
For that the simplest is to extend `AbstractMachine` so that it produces lines for the different motions.

In [ ]:
import libgcode.abstractmachine._
import libgcode._
import libgcode.extractor._
import Plane._
import scala.collection.mutable.ArrayBuffer
import scala.collection.mutable.StringBuilder

class Tracer extends AbstractMachine {
    
    import X3D._
    import X3DViewer._
    
    var circularInterpolationPrecision = 0.5 //roughly one line segment every 0.5 mm
    
    protected val toDraw = ArrayBuffer.empty[Tag]
    
    def resultByLines = toDraw.toSeq
    def result = group(toDraw.toSeq)
    
    override protected def linearMotion(x: Double, y: Double, z: Double,
                                        a: Double, b: Double, c: Double,
                                        f: Double) = {
        val x0 = getX
        val y0 = getY
        val z0 = getZ
        super.linearMotion(x,y,z,a,b,c,f)
        val x1 = getX
        val y1 = getY
        val z1 = getZ
        val points = coordinate( point := s"${x0} ${y0} ${z0}, ${x1} ${y1} ${z1}")
        val appear = if (f == maxFeed) lineAppearance("0.5 0.5 0.5") else lineAppearance("0 0 0")
        val line = shape(lineSet( vertexCount := "2" )( points ), appear )
        toDraw += line
    }
    
    override protected def circularMotion(_x: Double,_y: Double,_z: Double, // end position
                                          a: Double, b: Double, c: Double, // end orientation
                                          _i: Double,_j: Double,_k: Double, // center of rotation
                                          clockwise: Boolean, p: Int, f: Double) = { // number of turns and feedrate
        val coeff = if (useMillimeters) 1 else 25.4
        val x = coeff * _x
        val y = coeff * _y
        val z = coeff * _z
        val i = coeff * _i
        val j = coeff * _j
        val k = coeff * _k
        // get the center
        val cx = this.x + i
        val cy = this.y + j
        val cz = this.z + k
        val (radius, angle, offset, pitchOver2Pi) = plane match {
            case XY =>
                val radius = math.hypot(i, j)
                val angle = getRotationAngle(cx, cy, this.x, this.y, x, y, clockwise, p)
                val offset = math.atan2(getY - cy, getX - cx)
                val pitchOver2Pi = (z - this.z) / angle
                (radius, angle, offset, pitchOver2Pi)
            case ZX => 
                val radius = math.hypot(k, i)
                val angle = getRotationAngle(cz, cx, this.z, this.x, z, x, clockwise, p)
                val offset = math.atan2(getX - cx, getZ - cz)
                val pitchOver2Pi = (y - this.y) / angle
                (radius, angle, offset, pitchOver2Pi)
            case YZ => 
                val radius = math.hypot(j, k)
                val angle = getRotationAngle(cy, cz, this.y, this.z, y, z, clockwise, p)
                val offset = math.atan2(getZ - cz, getY - cy)
                val pitchOver2Pi = (x - this.x) / angle
                (radius, angle, offset, pitchOver2Pi)
        }
        val delta0 = circularInterpolationPrecision / 2 / math.Pi / radius
        val delta = if (clockwise) -delta0 else delta0
        assert(math.signum(delta) == math.signum(angle), s"delta: $delta, angle: $angle")
        def pointAt(a: Double) = {
            plane match {
                case XY => (cx + radius * math.cos(a+offset), cy + radius * math.sin(a+offset), this.z + pitchOver2Pi * a)
                case ZX => (cx + radius * math.sin(a+offset), this.y + pitchOver2Pi * a, cz + radius * math.cos(a+offset))
                case YZ => (this.x + pitchOver2Pi * a, cy + radius * math.cos(a+offset), cz + radius * math.sin(a+offset))
            }
        }
        var count = 2
        val points = new StringBuilder()
        var a = 0.0
        val (x0, y0, z0) = pointAt(a)
        points ++= s"${x0} ${y0} ${z0}"
        while ((a - angle).abs > delta.abs + 1e-5) {
            val a1 = a + delta
            count += 1
            val (x1, y1, z1) = pointAt(a1)
            points ++= s", ${x1} ${y1} ${z1}"
            a = a1
        }
        points ++= s", ${x} ${y} ${z}"
        val line = shape(lineSet( vertexCount := count.toString )( coordinate( point := points.toString) ), lineAppearance("0 0 0") )
        toDraw += line
        super.circularMotion(_x,_y,_z,a,b,c,_i,_j,_k,clockwise,p,f)
    }
    
}

def save(cmds: Seq[Command], fileName: String) = {
    Printer(cmds, fileName)
}

def display(cmds: Seq[Command], debug: Boolean = false) = {
    val tracer = new Tracer
    for (c <- cmds) {
        //println("processing " + c.toString)
        tracer.run(c)
    }
    if (debug) {
        val lines = tracer.resultByLines
        Text(lines.map(_.render).mkString("\n"))
    } else {
        val lines = tracer.result
        X3DViewer.display(lines)
    }
}

Some simple examples to test the tracing

In [ ]:
display(Seq(
    G(0, X(5), Y(5)),
    Empty(F(200)),
    G(1, Z(5)),
))

In [ ]:
display(Seq(
    G(0, X(5)),
    G(3, Y(5), X(0), R(5))
), false)

In [ ]:
def helix(x: Double, y: Double, radius: Double, pitch: Double, nbrTurns: Int, clockwise: Boolean = true) = {
    val buffer = scala.collection.mutable.ArrayBuffer.empty[Command]
    buffer += G(0, X(radius), Y(0))
    for (n <- 0 until nbrTurns) yield {
        val dir = if (clockwise) 2 else 3
        buffer += G(dir, X(-radius), I(-radius), Z(n*pitch + pitch/2))
        buffer += G(dir, X(radius), I(radius), Z(n*pitch + pitch))
    }
    buffer.toSeq
}

In [ ]:
display(helix(0, 0, 5, -2, 5))

In [ ]:
display(helix(0, 0, 5, 1, 5, false))

An more complex example to test the tracing

In [ ]:
object Box {
    
    // the stones to store (rectangular)
    val stoneLength = 60
    val stoneWidth = 6
    val stoneHeight = 25
    val numberOfStones = 4
    
    val cutterRadius = 3
    val depthOfPlunge = 4
    val feed = 200
    val plungeFeed = 100
    val clearanceZ = 5
    
    val boxLength = stoneLength + 2*cutterRadius + 8
    val boxWidth = stoneWidth + (numberOfStones-1)*(stoneWidth+4) + 8
    val boxCornerRadius = 4
    val boxHeight = 20
    val lidOverlap = 5
    val lidLip = 2
    val lidHeight = 10
    val stoneSpacing = 10
    val stoneToBorder = 4 + cutterRadius
    
    //machine position not including cutter
    def slot(xStart: Double, xStop: Double,
             yStart: Double, yStop: Double,
             zStart: Double, zStop: Double) = {
        var cmds = Seq( G(0, X(xStart), Y(yStart)) )
        
        def loop(z: Double) = {
            Seq(
                G(1, Z(z), F(plungeFeed)),
                G(1, X(xStop), F(feed)),
                G(1, Y(yStop)),
                G(1, X(xStart)),
                G(1, Y(yStart))
            )
        }
        var z = zStart
        while (z > zStop) {
            cmds = cmds ++ loop(z)
            z = z-depthOfPlunge
        }
        cmds = cmds ++ loop(zStop)
        cmds = cmds :+ G(0, Z(clearanceZ))
        cmds
    }
    
    //       p3_________p4
    //       /          \
    //    p2|            |p5
    //      |            |
    //      |            |
    //      |            |
    //    p1|            |p6
    //       \__________/
    //      p0         p7
    def roundedRectangleOuter(
            xMin: Double, xMax: Double, //actual dimension
            yMin: Double, yMax: Double, //actual dimension
            z: Double, cornerRadius: Double) = {
        val (x0, y0) = (xMin + cornerRadius, yMin - cutterRadius)
        val (x1, y1) = (xMin - cutterRadius, yMin + cornerRadius)
        val (x2, y2) = (xMin - cutterRadius, yMax - cornerRadius)
        val (x3, y3) = (xMin + cornerRadius, yMax + cutterRadius)
        val (x4, y4) = (xMax - cornerRadius, yMax + cutterRadius)
        val (x5, y5) = (xMax + cutterRadius, yMax - cornerRadius)
        val (x6, y6) = (xMax + cutterRadius, yMin + cornerRadius)
        val (x7, y7) = (xMax - cornerRadius, yMin - cutterRadius)
        Seq(
            G(0, X(x0), Y(y0)),
            G(1, Z(z), F(plungeFeed)),
            G(2, X(x1), Y(y1), R(cornerRadius+cutterRadius), F(feed)),
            G(1, X(x2), Y(y2)),
            G(2, X(x3), Y(y3), R(cornerRadius+cutterRadius)),
            G(1, X(x4), Y(y4)),
            G(2, X(x5), Y(y5), R(cornerRadius+cutterRadius)),
            G(1, X(x6), Y(y6)),
            G(2, X(x7), Y(y7), R(cornerRadius+cutterRadius)),
            G(1, X(x0), Y(y0)),
            G(0, Z(clearanceZ))
        )
    }
    
    def roundedRectangleInner(
            xMin: Double, xMax: Double, //actual dimension
            yMin: Double, yMax: Double, //actual dimension
            z: Double, cornerRadius: Double) = {
        if (cornerRadius > cutterRadius) {
            val (x0, y0) = (xMin + cornerRadius, yMin + cutterRadius)
            val (x1, y1) = (xMin + cutterRadius, yMin + cornerRadius)
            val (x2, y2) = (xMin + cutterRadius, yMax - cornerRadius)
            val (x3, y3) = (xMin + cornerRadius, yMax - cutterRadius)
            val (x4, y4) = (xMax - cornerRadius, yMax - cutterRadius)
            val (x5, y5) = (xMax - cutterRadius, yMax - cornerRadius)
            val (x6, y6) = (xMax - cutterRadius, yMin + cornerRadius)
            val (x7, y7) = (xMax - cornerRadius, yMin + cutterRadius)
            Seq(
                G(0, X(x0), Y(y0)),
                G(1, Z(z), F(plungeFeed)),
                G(1, X(x7), Y(y7), F(feed)),
                G(3, X(x6), Y(y6), R(cornerRadius - cutterRadius)),
                G(1, X(x5), Y(y5)),
                G(3, X(x4), Y(y4), R(cornerRadius - cutterRadius)),
                G(1, X(x3), Y(y3)),
                G(3, X(x2), Y(y2), R(cornerRadius - cutterRadius)),
                G(1, X(x1), Y(y1)),
                G(3, X(x0), Y(y0), R(cornerRadius - cutterRadius)),
                G(0, Z(clearanceZ))
            )
        } else {
            val (x0, y0) = (xMin + cutterRadius, yMin + cutterRadius)
            val (x1, y1) = (xMin + cornerRadius, yMax - cutterRadius)
            val (x2, y2) = (xMax - cutterRadius, yMax - cutterRadius)
            val (x3, y3) = (xMax - cornerRadius, yMin + cutterRadius)
            Seq(
                G(0, X(x0), Y(y0)),
                G(1, Z(z), F(plungeFeed)),
                G(1, X(x3), Y(y3), F(feed)),
                G(1, X(x2), Y(y2)),
                G(1, X(x1), Y(y1)),
                G(1, X(x0), Y(y0)),
                G(0, Z(clearanceZ))
            )
        }
    }
    
    def box = {
        var cmds = Seq[Command](G(0, Z(clearanceZ)))
        //careful here: 25 is the max depth for my 6mm endmill wich is the same as stoneHeight
        //removing the stock around the box
        for (depth <- Seq(-stoneHeight/2, -stoneHeight);
             i <- Seq(4, 1)) {
            cmds = cmds ++ roundedRectangleOuter(-i, (numberOfStones+1)*stoneSpacing+i,
                                                 -i, stoneLength + 2*stoneToBorder+i,
                                                 depth,6+i)
        }
        //outer
        cmds = cmds ++ roundedRectangleOuter(0, (numberOfStones+1)*stoneSpacing,
                                             0, stoneLength + 2*stoneToBorder,
                                             -stoneHeight,6)
        // lip
        cmds = cmds ++ roundedRectangleOuter(lidLip, (numberOfStones+1)*stoneSpacing-lidLip,
                                             lidLip, stoneLength + 2*stoneToBorder -lidLip,
                                             -5, 4)
        // slots for the stones
        for (i <- 1 to numberOfStones) {
            cmds = cmds ++ slot(i*stoneSpacing-1, i*stoneSpacing+1,
                                stoneToBorder, stoneToBorder+stoneLength,
                                -depthOfPlunge, -stoneHeight+2)
        }
        cmds
    }
    
    def lid = {
        var cmds = Seq[Command](G(0, Z(clearanceZ)))
        //outer
        cmds = cmds ++ roundedRectangleOuter(0, (numberOfStones+1)*stoneSpacing,
                                             0, stoneLength + 2*stoneToBorder,
                                             -10,6)
        // clear inner up to the lip
        var i = math.min(stoneLength + 2*stoneToBorder - 2*lidLip, (numberOfStones+1)*stoneSpacing-2*lidLip) / 2.0 - cutterRadius* 1.5
        while (i > 0) {
            cmds = cmds ++ roundedRectangleInner(lidLip + i, (numberOfStones+1)*stoneSpacing - lidLip -i,
                                                 lidLip + i, stoneLength + 2*stoneToBorder - lidLip -i,
                                                 -5,4)
            i -= cutterRadius * 1.5 ///0.75 step over
        }
        cmds = cmds ++ roundedRectangleInner(lidLip, (numberOfStones+1)*stoneSpacing-lidLip,
                                             lidLip, stoneLength + 2*stoneToBorder - lidLip,
                                             -5, 4)
        // slots for the stones
        for (i <- 1 to numberOfStones) {
            cmds = cmds ++ slot(i*stoneSpacing-1, i*stoneSpacing+1,
                                stoneToBorder, stoneToBorder+stoneLength,
                                -5-depthOfPlunge, -8)
        }
        cmds
    }
    
}

In [ ]:
display(Box.box)

In [ ]:
display(Box.lid)

In [ ]:
save(Box.box, "box.nc")
save(Box.lid, "lid.nc")